# Part I. ETL Pipeline for Pre-Processing the Files

##  PRE-PROCESSING THE FILES

#### Import Python packages 

The required Python packages are imported

In [119]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

Filepaths to processing the original csv files are created and printed

In [120]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

A full csv file is created with all the data for further use in Cassandra

In [121]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


The number of final rows is printed and checked. Should be 6821.

In [122]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The final CSV file titled <font color=red>event_datafile_new.csv</font> is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Starting up Apache Cassandra NoSQL database

#### Creating a Cluster

First a connection is created to connect to a Cassandra Cluster

In [123]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

A new Keyspace is created with the name 'udacity'

In [124]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

The Session is connected to the new 'udacity' Keyspace

In [125]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


#### Queries are first defined for the Data Model

The following queries are the basis for the Apache Cassandra data model:

 1. _Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4_


 2. _Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182_
    

 3. _Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'_




#### First Query

The following CQL query is used to the data modeling: 
    
    select artist, song, length from artist_song_length_in_session where sessionId=338 and itemInSession=4
    
This means we need a composite primary key with sessionId as partition key and iteminSession as clustering column for a fast and efficient query execution

In [126]:
query = "CREATE TABLE IF NOT EXISTS artist_song_length_in_session "
query = query + "(sessionId int, \
                itemInSession int, \
                artist text, \
                song text, \
                length decimal, \
                PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

The data is filled into the precreated table in the correct order

In [127]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO artist_song_length_in_session (\
                sessionId,\
                itemInSession, \
                artist, \
                song, \
                length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

The select is run as a cross check of functionality

In [128]:
query = "select artist, song, length from artist_song_length_in_session where sessionId=338 and itemInSession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length,)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### Second Query

The following CQL query is used to the data modeling: 
    
    select artist, song, firstname, lastname from song_playlist_in_session where userId=10 and sessionId=182
    
This means we need a composite primary key with sessionId as partition key and userId and itemInSession as clustering columns for a fast and efficient query execution. The itemInSession is needed for ordering purposes.

In [129]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_in_session "
query = query + "(sessionId int,\
                userId int, \
                itemInSession int, \
                artist text, \
                song text, \
                firstName text, \
                lastName text, \
                PRIMARY KEY (sessionId, userid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

The data is filled into the precreated table in the correct order

In [130]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO song_playlist_in_session (\
                sessionId,\
                userId, \
                itemInSession, \
                artist, \
                song, \
                firstName, \
                lastName \
                )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[10]), int(line[3]),  line[0], line[9], line[1], line[4] ))

The select is run as a cross check of functionality

In [131]:
query_2 = "select artist, song, firstname, lastname from song_playlist_in_session where userId=10 and sessionId=182"

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname,)
                   

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Third Query

The following CQL query is used to the data modeling: 
    
    select firstname, lastname from userinfo_by_song where song='All Hands Against His Own'
    
 This means we need a composite primary key with song as partition key and userinfo columns as clustering columns for a fast and efficient query execution. 

In [132]:
query = "CREATE TABLE IF NOT EXISTS userinfo_by_song "
query = query + "(song text, \
                firstName text, \
                lastName text, \
                userId int, \
                PRIMARY KEY (song, firstName, lastName, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

The data is filled into the precreated table in the correct order

In [133]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO userinfo_by_song (\
                song, \
                firstName, \
                lastName, \
                userId)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], line[1], line[4], int(line[10])))

The select is run as a cross check of functionality

In [134]:
query_3 = "select firstname, lastname from userinfo_by_song where song='All Hands Against His Own'"

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname,)
                   

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [135]:
query = "drop table artist_song_length_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [136]:
query = "drop table song_playlist_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [137]:
query = "drop table userinfo_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [138]:
session.shutdown()
cluster.shutdown()